In [19]:
@tdmclient.notebook.sync_var
def set_speed(left, right):
    global motor_left_target, motor_right_target
    motor_left_target = left
    motor_right_target = right

In [216]:
# -*- coding: utf-8 -*-
"""
Created on Sun Nov 28 17:03:23 2021

@author: henry papapatos
"""
import numpy as np
import sys
#sys.path.append(r'C:\Users\Usuario\OneDrive - epfl.ch\Documents\EPFL\Basics of mobile robotics\PROJET\Github\Mobile_robotics')
import math
import cv2 
import time
import importlib
import filters 
import computer_vision
import path_planning

TRESH_DIST = 10 #mm
CAMERA = 0 # Camera
SPEED_TO_MMS = 0.3436 #150 speed
YAW_TO_MMS = 0.06086 # -100 100

def main():
         
    try:              
        frame_init = computer_vision.get_image(cap)
        
        # we need to wait a bit otherwise the image is yellow 
        time.sleep(1)
        
        print("take the rigth image")
        frame_init = computer_vision.get_image(cap)
        
        #frame_init = cv2.cvtColor(frame_init, cv2.COLOR_BGR2RGB)
        frame_init_crop = cv2.resize(frame_init,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)

        cv2.imshow('frame', frame_init_crop)
        
        cv2.waitKey(0)
        
        cv2.imwrite(r'C:\Users\papad\OneDrive\Images\Pellicule\img.jpg', frame_init) #POUR TUNING PAR ELIOTT
        
        # Extract vertexes, goals, thymio's start position and orientation from first frame
        start_pos, obst_vertexes, goals_pos, px_to_mm = computer_vision.Init(frame_init)
        
        print("Px_to_mm= ", px_to_mm)
        
        px_to_mm = 0.64
        
        computer_vision.display_obstacle(frame_init, start_pos, goals_pos, obst_vertexes)
        
        frame_init_crop = cv2.resize(frame_init,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
        
        cv2.imshow('frame', frame_init_crop)
        
        cv2.waitKey(0)
        
        cv2.imwrite(r'C:\Users\papad\OneDrive\Images\Pellicule\img_obstacle.jpg', frame_init) #POUR TUNING PAR ELIOTT
        
        goal_list = path_planning.get_optimal_path(start_pos, goals_pos, obst_vertexes, 
                                                   px_to_mm, draw = True, image = frame_init)
        
        frame_init_crop = cv2.resize(frame_init,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
        
        cv2.imwrite(r'C:\Users\papad\OneDrive\Images\Pellicule\img_path.jpg', frame_init) #POUR TUNING PAR ELIOTT

        cv2.imshow('frame', frame_init_crop)
        
        cv2.waitKey(0)
        
        frame = computer_vision.get_image(cap)
        
        # The estimated state vector at time t-1 in the global reference frame.
        # [x_t_minus_1, y_t_minus_1, yaw_t_minus_1]
        x_est_t_minus_1, hidden_cam, mask_frame = computer_vision.vision(frame, px_to_mm)
        # x_est_t_minus_1 = np.array([0.0,0.0,0.0])
        
        # The control input vector at time t-1 in the global reference frame.
        # [v, yaw_rate]
        u_t_minus_1 = np.array([0 ,0])
         
        # State covariance matrix P_t_minus_1
        P_t_minus_1 = np.array([[0.1,  0,   0],
                                [  0,0.1,   0],
                                [  0,  0, 0.1]])
    
    
        #update le dt car sinon kalman marche pas
        dt = 0.1 # EN SECONDES POUR KALMAN!!!!! same as PI_CLOCK of pid defined in filters?
        sum_error = 0 
        alt_error_pid = 0
        
        time.sleep(0.1)
        
        while len(goal_list)!=0:
            
            print("goal is: ", goal_list[0])
            
            previous = time.time()
            
            frame = computer_vision.get_image(cap)
            
#             cv2.imshow('running frame', frame)

                
            ##### APPELER get_vision_position##########
            obs_vector_z_t, hidden_cam, mask_frame = computer_vision.vision(frame, px_to_mm)
            
            
            computer_vision.display_obstacle(frame, start_pos, goals_pos, obst_vertexes)
            
            computer_vision.display_pos(frame,obs_vector_z_t, px_to_mm, hidden_cam, is_from_camera = True)
            
            
#             print(f'Timestep measurement={obs_vector_z_t}')
    
            # Run the Extended Kalman Filter and store the 
            # near-optimal state and covariance estimates
            optimal_state_estimate_t, covariance_estimate_t = filters.ekf(
                obs_vector_z_t, # Most recent sensor measurement
                x_est_t_minus_1, # Our most recent estimate of the state
                u_t_minus_1, # Our most recent control input
                P_t_minus_1, # Our most recent state covariance matrix
                dt,hidden_cam) # Time interval
            
            computer_vision.display_pos(frame, optimal_state_estimate_t, px_to_mm, 0, is_from_camera = False)
            computer_vision.display_pos(frame, goal_list[0], px_to_mm, 0, is_from_camera = 2)
            
            frame = cv2.resize(frame,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
            cv2.imshow('running frame', frame)
            if cv2.waitKey(1) == ord('q'):
                break
            
            mask_frame = cv2.resize(mask_frame,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
            cv2.imshow('mask frame', mask_frame)
            if cv2.waitKey(1) == ord('q'):
                break
                
            if np.linalg.norm(np.array([optimal_state_estimate_t[0],optimal_state_estimate_t[1]])-np.array(goal_list[0])) < TRESH_DIST:
                goal_list.pop(0)
                
            # Get ready for the next timestep by updating the variable values
            x_est_t_minus_1 = optimal_state_estimate_t
            P_t_minus_1 = covariance_estimate_t
            
            #check if obstacle in coming
            
            #if no
            ######APPELER PID##########
            print()
            sum_error, alt_error_pid, v_l,v_d = filters.pid(optimal_state_estimate_t,goal_list[0],sum_error, alt_error_pid,dt, True)
#             v_l = 50
#             v_d = 50
            v = (v_l + v_d)*SPEED_TO_MMS/2
            yaw = (v_d - v_l)*YAW_TO_MMS
            u_t_minus_1 = [v, yaw]; # delta_v returned by PID
#             u_t_minus_1 = 0;
            set_speed(int(v_l), int(v_d))

    
    
            #if yes, obstacle avoidance 
            
            actual = time.time()
            diff = actual-previous
            print("elapsed time is: ", diff)
            if diff<dt:
                time.sleep(dt-diff)
            
    finally: 
        set_speed(0, 0)
        cv2.destroyAllWindows()



In [124]:
print('Connecting to camera...')
cap=cv2.VideoCapture(CAMERA) # Either 0 or 1, front camera or external cam
print('Camera connected !')

Connecting to camera...
Camera connected !


In [125]:
#get the full quality of the camera
print("setting parameters...")
cap.set(3,1920) 
print("first parameter set")
cap.set(4,1080)
print("second parameter set")

setting parameters...
first parameter set
second parameter set


In [10]:
import tdmclient.notebook
print("connecting to thymio")
await tdmclient.notebook.start()

connecting to thymio


In [217]:
importlib.reload(filters)
importlib.reload(path_planning)
importlib.reload(computer_vision)

<module 'computer_vision' from 'C:\\Users\\papad\\OneDrive\\Documents\\MA1\\Basic_of_mobile_robotics\\project\\Mobile_robotics\\computer_vision.py'>

In [223]:
main()

take the rigth image
Px_to_mm=  0.48486691339264704


100%|██████████| 2/2 [00:00<00:00, 282.08it/s]


goal is:  [200.32  14.08]

pos robot is:  [202.68   13.88   71.729]
y: 260.6804761904762
x: 111.56047619047618
Robot_angle: 71.72945024996552
angle goal:  66.83107550682243
Error_angle: 4.898374743143094
Moving and turning, speed l and r: 90.20325051371381 109.79674948628619
Sum_error, Alt_error_pid: 4.898374743143094 0
elapsed time is:  0.14801979064941406
goal is:  [314.24 274.56]

pos robot is:  [203.945  14.653  71.351]
y: 259.90695087078177
x: 110.29529628730205
Robot_angle: 71.35074719745879
angle goal:  67.00535345151161
Error_angle: 4.345393745947177
Moving and turning, speed l and r: 91.30921250810565 108.69078749189435
Sum_error, Alt_error_pid: 9.243768489090272 4.898374743143094
elapsed time is:  0.09877300262451172
goal is:  [314.24 274.56]

pos robot is:  [204.05   15.285  71.704]
y: 259.2746718939282
x: 110.19015825646119
Robot_angle: 71.7036536281215
angle goal:  66.97480322480784
Error_angle: 4.728850403313658
Moving and turning, speed l and r: 90.54229919337268 109.457


pos robot is:  [219.916  63.951  67.636]
y: 210.6086954737264
x: 94.32361728230964
Robot_angle: 67.63642609769617
angle goal:  65.87420913389656
Error_angle: 1.7622169637996166
Moving and turning, speed l and r: 96.47556607240077 103.52443392759923
Sum_error, Alt_error_pid: 50 1.7121126672907678
elapsed time is:  0.07315349578857422
goal is:  [314.24 274.56]

pos robot is:  [221.076  66.573  67.89 ]
y: 207.98688330073298
x: 93.16408069044593
Robot_angle: 67.89036053451147
angle goal:  65.87084238244213
Error_angle: 2.019518152069338
Moving and turning, speed l and r: 95.96096369586132 104.03903630413868
Sum_error, Alt_error_pid: 50 1.7622169637996166
elapsed time is:  0.06700968742370605
goal is:  [314.24 274.56]

pos robot is:  [221.985  70.02   67.99 ]
y: 204.5399302941167
x: 92.25452170039833
Robot_angle: 67.98974739429276
angle goal:  65.7229928450277
Error_angle: 2.266754549265059
Moving and turning, speed l and r: 95.46649090146988 104.53350909853012
Sum_error, Alt_error_pid: 50


pos robot is:  [247.761 133.459  66.36 ]
y: 141.10082125936685
x: 66.47933410629673
Robot_angle: 66.35999720896368
angle goal:  64.77263648359488
Error_angle: 1.5873607253687965
Moving and turning, speed l and r: 96.8252785492624 103.1747214507376
Sum_error, Alt_error_pid: 50 1.8271238187649317
elapsed time is:  0.07427787780761719
goal is:  [314.24 274.56]

pos robot is:  [248.87  136.653  66.235]
y: 137.90664945143612
x: 65.37042977647755
Robot_angle: 66.23513240751471
angle goal:  64.63813974356582
Error_angle: 1.596992663948896
Moving and turning, speed l and r: 96.80601467210221 103.19398532789779
Sum_error, Alt_error_pid: 50 1.5873607253687965
elapsed time is:  0.08242225646972656
goal is:  [314.24 274.56]

pos robot is:  [250.136 140.006  65.983]
y: 134.55363236156253
x: 64.10362587644059
Robot_angle: 65.98284378688508
angle goal:  64.52608022173395
Error_angle: 1.4567635651511353
Moving and turning, speed l and r: 97.08647286969773 102.91352713030227
Sum_error, Alt_error_pid: 

...


pos robot is:  [401.388  18.719 195.019]
y: -4.638658778374792
x: -201.06797701582252
Robot_angle: -164.98132816618906
angle goal:  -178.67841494012345
Error_angle: 13.697086773934387
Moving and turning, speed l and r: 72.60582645213123 127.39417354786877
Sum_error, Alt_error_pid: 50 11.85200593517439
elapsed time is:  0.06422162055969238
goal is:  [200.32  14.08]

pos robot is:  [396.43   20.062 193.115]
y: -5.982266030131326
x: -196.10968881725853
Robot_angle: -166.88481085701758
angle goal:  -178.25275160858834
Error_angle: 11.367940751570757
Moving and turning, speed l and r: 77.26411849685849 122.73588150314151
Sum_error, Alt_error_pid: 50 13.697086773934387
elapsed time is:  0.052834510803222656
goal is:  [200.32  14.08]

pos robot is:  [393.954  18.957 192.366]
y: -4.87680869404595
x: -193.6344890662735
Robot_angle: -167.63353429988354
angle goal:  -178.5572740727792
Error_angle: 10.923739772895658
Moving and turning, speed l and r: 78.15252045420868 121.84747954579132
Sum_erro


pos robot is:  [323.562  21.89  183.416]
y: -7.809746496332339
x: -123.2424016100249
Robot_angle: -176.58444476406754
angle goal:  -176.37407260300174
Error_angle: -0.21037216106580559
Moving and turning, speed l and r: 100.42074432213161 99.57925567786839
Sum_error, Alt_error_pid: 48.88560176259361 -0.9040260763405854
elapsed time is:  0.045313119888305664
goal is:  [200.32  14.08]

pos robot is:  [320.832  22.186 183.625]
y: -8.106384797963871
x: -120.51220106568144
Robot_angle: -176.3748424085906
angle goal:  -176.15173395818678
Error_angle: -0.22310845040382787
Moving and turning, speed l and r: 100.44621690080766 99.55378309919234
Sum_error, Alt_error_pid: 48.66249331218978 -0.21037216106580559
elapsed time is:  0.05813264846801758
goal is:  [200.32  14.08]

pos robot is:  [317.286  22.371 182.737]
y: -8.29102771272783
x: -116.96572267953701
Robot_angle: -177.26292105216808
angle goal:  -175.9454133364974
Error_angle: -1.317507715670672
Moving and turning, speed l and r: 102.6350

elapsed time is:  0.06391048431396484
goal is:  [200.32  14.08]

pos robot is:  [252.164  21.471 185.849]
y: -7.391196309086096
x: -51.843726817730015
Robot_angle: -174.1507400630931
angle goal:  -171.88619877074936
Error_angle: -2.2645412923437505
Moving and turning, speed l and r: 104.5290825846875 95.4709174153125
Sum_error, Alt_error_pid: 4.3612000246411355 -2.683663515389071
elapsed time is:  0.05172610282897949
goal is:  [200.32  14.08]

pos robot is:  [248.513  20.178 187.23 ]
y: -6.097978119938345
x: -48.192803332024
Robot_angle: -172.76996052858135
angle goal:  -172.78851908121047
Error_angle: 0.01855855262911632
Moving and turning, speed l and r: 99.96288289474177 100.03711710525823
Sum_error, Alt_error_pid: 4.379758577270252 -2.2645412923437505
elapsed time is:  0.07535815238952637
goal is:  [200.32  14.08]

pos robot is:  [243.591  19.452 188.798]
y: -5.3715882626736455
x: -43.271385351720085
Robot_angle: -171.2019404291653
angle goal:  -172.92366353482882
Error_angle: 1.72

IndexError: list index out of range

Exception in thread Thread-9:
Traceback (most recent call last):
  File "C:\Users\papad\anaconda3\envs\mobile_robotique\lib\threading.py", line 973, in _bootstrap_inner
Exception in thread Exception in thread Thread-17:
Traceback (most recent call last):
  File "C:\Users\papad\anaconda3\envs\mobile_robotique\lib\threading.py", line 973, in _bootstrap_inner
Exception in thread Thread-15:
Traceback (most recent call last):
  File "C:\Users\papad\anaconda3\envs\mobile_robotique\lib\threading.py", line 973, in _bootstrap_inner
Exception in thread Thread-13:
Traceback (most recent call last):
  File "C:\Users\papad\anaconda3\envs\mobile_robotique\lib\threading.py", line 973, in _bootstrap_inner
Thread-11:
Traceback (most recent call last):
  File "C:\Users\papad\anaconda3\envs\mobile_robotique\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\papad\anaconda3\envs\mobile_robotique\lib\site-packages\tdmclient\tcp.py", line 65, in run
    self.run()
  File "C:\Us

In [123]:
cap.release()

In [221]:
set_speed(0, 0)